# Shape classification

The notebooks in this folder replicate the experiments as performed for [CNNs on Surfaces using Rotation-Equivariant Features](https://doi.org/10.1145/3386569.3392437).

The current notebook replicates the shape classification experiments from section `5.2 Comparisons`.

## Imports
We start by importing dependencies.

In [4]:
# File reading and progressbar
import os.path as osp
import progressbar

# PyTorch and PyTorch Geometric dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn.inits import zeros

# Harmonic Surface Networks components
# Layers
from nn import HarmonicResNetBlock, ParallelTransportPool, ParallelTransportUnpool, ComplexLin, ComplexNonLin
# Utility functions
from utils.harmonic import magnitudes
# Rotated MNIST dataset
from datasets import MNISTSphere
# Transforms
from transforms import (HarmonicPrecomp, VectorHeat, MultiscaleRadiusGraph, 
                        ScaleMask, FilterNeighbours, NormalizeArea, NormalizeAxes, Subsample)

## Settings
Next, we set a few parameters for our network. You can change these settings to experiment with different configurations of the network. Right now, the settings are set to the ones used in the paper.

In [1]:
# Maximum rotation order for streams
max_order = 1
# Number of rings in the radial profile
n_rings = 6
# Number of filters per block
nf = [16, 32]
# Ratios used for pooling
ratios=[1, 0.25]
# Radius of convolution for each scale
radii = [0.2, 0.4]
# Number of datasets per batch
batch_size = 1

## Dataset
To get our dataset ready for training, we need to perform the following steps:
1. Provide a path to load and store the dataset.
2. Define transformations to be performed on the dataset:
    - A transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
    - A transformation that masks the edges and vertices per scale and precomputes convolution components.
3. Assign and load the datasets.

In [ ]:
# 1. Provide a path to load and store the dataset.
# Make sure that you have created a folder 'data' somewhere
# and that you have downloaded and moved the raw datasets there
path = osp.join('data', 'shrec')

# 2. Define transformations to be performed on the dataset:
# Transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
pre_transform = T.Compose((
    NormalizeArea(),
    MultiscaleRadiusGraph(ratios,radii, loop=True, flow='target_to_source'),
    VectorHeat()
))
# Apply a random scale and random rotation to each shape
transform = T.Compose((
    T.RandomScale((0.85, 1.15)),
    T.RandomRotate(45, axis=0),
    T.RandomRotate(45, axis=1),
    T.RandomRotate(45, axis=2))
)

# Transformations that masks the edges and vertices per scale and precomputes convolution components.
scale0_transform = T.Compose((
    ScaleMask(0),
    FilterNeighbours(radius[0]),
    HarmonicPrecomp(n_rings, max_order, max_r=radius[0]))
)
scale1_transform = T.Compose((
    ScaleMask(1),
    FilterNeighbours(radius[1]),
    HarmonicPrecomp(n_rings, max_order, max_r=radius[1]))
)

# 3. Assign and load the datasets.
test_dataset = Shrec16(path, False, pre_transform=pre_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
train_dataset = Shrec16(path, True, pre_transform=pre_transform, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

n_classes = train_dataset.num_classes

## Network architecture
Now, we create the network architecture by creating a new `nn.Module`, `Net`. We first setup each layer in the `__init__` method of the `Net` class and define the steps to perform for each batch in the `forward` method. We use the left part of the U-ResNet architecture, with half of the ResNet blocks from the following figure:

<img src="img/resnet_architecture.png" width="800px" />

Let's get started!

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.lin0 = nn.Linear(3, nf[0])

        # ResNet Block 1
        self.resnet_block11 = HarmonicResNetBlock(nf[0], nf[0], max_order, n_rings, prev_order=0)

        # Pool
        self.pool = ParallelTransportPool(1, scale1_transform)

        # ResNet Block 2
        self.resnet_block21 = HarmonicResNetBlock(nf[0], nf[1], max_order, n_rings)

        # Final Harmonic Convolution
        # We set offset to False, 
        # because we will only use the radial component of the features after this
        self.conv_final = HarmonicConv(nf[1], n_classes, max_order, n_rings, offset=False)
        
        self.bias = nn.Parameter(torch.Tensor(n_classes))
        zeros(self.bias)


    def forward(self, data):
        x = data.pos
        
        # Linear transformation from input descriptors to nf[0] features
        x = F.relu(self.lin0(x))
        # Convert input features into complex numbers
        x = torch.stack((x, torch.zeros_like(x)), dim=-1).unsqueeze(1)

        # ResNet Block 1
        # Select only the edges and precomputed components of the first scale
        data_scale0 = scale0_transform(data)
        attributes = (data_scale0.edge_index, data_scale0.precomp, data_scale0.connection)
        x = self.resnet_block11(x, *attributes)

        # Pooling
        # Apply parallel transport pooling
        x, data, data_pooled = self.pool(x, data)
        
        # ResNet Block 2
        # Store edge_index and precomputed components of the second scale
        attributes_pooled = (data_pooled.edge_index, data_pooled.precomp, data_pooled.connection)
        x = self.resnet_block21(x, *attributes_pooled)

        x = self.conv_final(x, *attributes_pooled)
        
        # Take radial component from features and sum streams
        x = magnitudes(x, keepdim=False)
        x = x.sum(dim=1)

        # Global mean pool
        x = torch.mean(x, dim=0, keepdim=True)
        x = x + self.bias
        return F.log_softmax(x, dim=1)

## Training

Phew, we're through the hard part. Now, let's get to training. First, move the network to the GPU and setup an optimizer.

In [ ]:
# We want to train on a GPU. It'll take a long time on a CPU
device = torch.device('cuda')
# Move the network to the GPU
model = Net().to(device)
# Set up the ADAM optimizer with learning rate of 0.0076 (as used in H-Nets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Next, define a training and test function.

In [ ]:
def train(epoch):
    # Set model to 'train' mode
    model.train()

    if epoch == 30:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

    for data in progressbar.progressbar(train_loader):
        # Move training data to the GPU and optimize parameters
        optimizer.zero_grad()
        F.nll_loss(model(data.to(device)), data.y).backward()
        optimizer.step()

Train for 100 epochs.

In [ ]:
for epoch in range(50):
    print('Epoch {}'.format(epoch))
    train(epoch)

## Testing
Finally, we test our model on the test dataset. Setup a test function:

In [ ]:
def test():
    # Set model to 'evaluation' mode
    model.eval()
    correct = 0
    total_num = 0
    for i, data in progressbar.progressbar(enumerate(test_loader)):
        pred = model(data.to(device)).max(1)[1]
        correct += pred.eq(data.y).sum().item()
        total_num += 1
    return correct / total_num

And return the accuracy on the test set:

In [ ]:
test_acc = test()
print('Test: {:.6f}'.format(test_acc))